In [0]:
import os

DEBUG = False

GALAXY = 'm31'
GALAXYFULLNAME = 'm31'
NVISITS = 1

DATA_DIR = "/raid/pfs" #os.environ['PFS_DATA_DIR']
OUTPUT_PATH = DATA_DIR+f'/data/targeting/dSph/{GALAXYFULLNAME}/netflow/{GALAXYFULLNAME}_{NVISITS}'

# Plot the instrument configuration

Given a netflow run, read back the config files and plot the instrument configuration. This currently consists of the cobra groups that set limits of how many sky fibers to allocate uniformly along the slits and how many flux standards to allocate evenly over the field of view.

In [0]:
import os, sys
from glob import glob
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.m31 import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
wcs = WcsProjection(Pointing(galaxy.pos), proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load the configuration

In [0]:
from pfs.ga.targeting.config import NetflowConfig
from pfs.ga.targeting.io import ObservationSerializer

In [0]:
# Load the configuration
fn = glob(os.path.join(OUTPUT_PATH, 'ga-netflow*.config'))[0]
config = NetflowConfig.from_file(fn, format='.yaml', ignore_collisions=True)

print(config.targets.keys())

In [0]:
pfi = SubaruPFI(instrument_options=config.instrument_options)

# Load the assignments

In [0]:
from pfs.ga.targeting.io import DataFrameSerializer, ObservationSerializer

In [0]:
assignments = DataFrameSerializer().read(f'{OUTPUT_PATH}/{GALAXY}_assignments.feather')

print(assignments.shape)
for c in assignments.columns:
    print(c, assignments[c].dtype)

In [0]:
assignments['pointing_idx'].unique(), assignments['visit_idx'].unique()

# Plot the cobra groups

In [0]:
from pfs.datamodel import TargetType

In [0]:
for k, options in config.netflow_options.cobra_groups.items():
    print(k, options.target_classes, len(options.groups), np.max(options.groups) + 1, options.min_targets, options.max_targets)

In [0]:
# Active configuration options
config.debug_options.ignore_cobra_group_minimum, config.debug_options.ignore_cobra_group_maximum

In [0]:
def plot_instrument(pointing_idx, visit_idx, cobra_group):
     cmap = plt.get_cmap('gist_rainbow')

     f, axs = plt.subplots(1, 3, figsize=(12, 3), dpi=240)

     cobra_labels = np.array(config.netflow_options.cobra_groups[cobra_group].groups)          # <<<<---- rename to labels everywhere
     target_classes = config.netflow_options.cobra_groups[cobra_group].target_classes
     cobra_group_count = cobra_labels.max() + 1
     spectrograph_count = np.unique(pfi.fiber_map.spectrographId).max()

     # print(np.unique(cobra_labels))
     # print(target_classes)

     colors = np.random.permutation(np.arange(cobra_group_count))

     ################
     # Plot the focal plane

     mask = (assignments['visit_idx'] == visit_idx) & (assignments['cobraid'] != -1)
     for ax, coloring in zip(axs[:2], ['spectrograph', 'cobra_group']):

          if coloring == 'spectrograph':
               c = np.array(assignments[mask].sort_values(['cobraid'])['spectrographid'])
          else:
               c = colors[cobra_labels] / cobra_group_count

          pfi.plot_cobras(ax, fp, data=c, cmap=cmap, alpha=0.3)

          # Plot the assigned targets
          mask = (assignments['visit_idx'] == visit_idx) \
               & (assignments['class'].isin(target_classes))

          x = assignments['fp_x'][mask]
          y = assignments['fp_y'][mask]
          fp.plot(ax, x, y, native_frame='pixel', marker='o', ms=1)

          ax.set_xlim(-250, 250)
          ax.set_ylim(-250, 250)

          fp.apply(ax)

     ##########
     # Plot the slits

     # Iterate over the spectrographs
     for s in np.arange(np.unique(pfi.fiber_map.spectrographId).max()):
          slit_mask = (pfi.fiber_map.spectrographId == s + 1) \
                    & (pfi.fiber_map.cobraId != 65535)
     
          # Plot the fibers along the slit
          slit_cobras = pfi.fiber_map.cobraId[slit_mask]
          
          x = pfi.fiber_map.fiberHoleId[slit_mask]
          y = np.zeros_like(x) + s
          c = colors[cobra_labels[slit_cobras - 1]] / cobra_group_count

          axs[2].scatter(x, y, s=2, c=cmap(c), alpha=0.1)

          mask = (assignments['visit_idx'] == visit_idx) \
               & (assignments['spectrographid'] == s + 1) \
               & (assignments['class'].isin(config.netflow_options.cobra_groups[cobra_group].target_classes)) \

          x = assignments['fiberholeid'][mask]
          y = np.zeros_like(x) + s + 0.25
          
          axs[2].plot(x, y, 'o', ms=1, c='k')

     f.suptitle(f'Pointing ID {pointing_idx}, visit ID {visit_idx}, Cobra group: `{cobra_group}`, coloring by {coloring}')
     f.show()

In [0]:
for cobra_group in config.netflow_options.cobra_groups.keys():
    for pidx in assignments['pointing_idx'].unique():
        for vidx in assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique():
            # print(p.ra, p.dec, p.posang)
            plot_instrument(pidx, vidx, cobra_group)